In [ ]:
import numpy as np
import pandas 
import os 

THRESHOLD = 0.3
BLACK_LIST = []

def get_dpr(df, want):
    a = df.iloc[:, :-1]
    b = df.iloc[:, 1:]
    b.columns = a.columns = range(df.shape[1] - 1)
    r = (b - a) / a
    p = 1 / np.linalg.norm(r, axis=1)
    r = (r.T * p).T.values
    l = r.shape[0]

    correlation = np.ones([l, l])
    for i in range(l):
        for j in range(i + 1, l):
            s = (r[i] * r[j]).sum()
            correlation[i][j] = s
            correlation[j][i] = s
    correlation[np.abs(correlation) > THRESHOLD] = 0
    correlation[np.logical_and(np.abs(correlation) <= THRESHOLD, correlation != 0)] = 1

    return (np.concatenate((np.ones([r.shape[0], 1]) * want,
                            np.add.accumulate(r, axis=1) + np.ones(r.shape) * want), axis=1) / want,p,r,
            correlation)

def find_eligible(got, d, eligible):
    if len(got) == 0: return list(range(d.shape[0]))
    origin = eligible[got[0]]
    for i in range(1, len(got)):
        origin = np.logical_and(eligible[got[i]], origin)
    ret = np.where(origin)[0].tolist()
    return [i for i in ret if i not in BLACK_LIST]

def factorial_random(gots, d, eligible, repeat, want):
    if gots is None or len(gots) == 0:
        gots = []
        f = find_eligible([], d, eligible)
        for j in range(repeat):
            c = np.random.choice(f, 1)[0]
            gots.append([c])

    toContinue = []
    toKeep = []
    for got in gots:
        if len(got) == want:
            toKeep.append(got)
        else:
            toContinue.append(got)

    if len(toContinue) == 0:
        return toKeep

    fines = []
    for got in toContinue:
        fines.append(find_eligible(got, d, eligible))
        
    nextGots = []
    for i in range(len(toContinue)):
        for j in range(repeat):
            if len(fines[i]) > 0:
                c = np.random.choice(fines[i], 1)[0]
                nextGots.append(toContinue[i] + [c])

    return factorial_random(toKeep + nextGots, d, eligible, repeat, want)


def fitLossAbsMax(s):
    slope, b = np.polyfit(np.arange(len(s)), s, 1)
    line = np.arange(len(s)) * slope + b
    return np.abs(s / line - 1).max(), slope, b


def BF(df, want, times):
    if os.path.exists('lasttime.npy'):
        lasttime = np.load('lasttime.npy', allow_pickle=True)
    else: lasttime = None
    d, p, r, eligible = get_dpr(df, want)
    for i in range(times):
        ls = factorial_random([], d, eligible, 2, want)
        thistime = []
        for l in ls:
            s = d[l].sum(axis=0) / len(l)
            loss, slope, b = fitLossAbsMax(s)
            thistime.append([l, loss, slope, slope * (len(s)-1)])
        thistime = pandas.DataFrame(thistime).sort_values(1)
        if lasttime is not None:    
            thistime = thistime[thistime[1] < lasttime[-1][1]]
            lasttime = pandas.concat([pandas.DataFrame(lasttime), thistime], axis=0).sort_values(1).values[:5000]
        else: lasttime = thistime.values
    np.save('lasttime.npy', lasttime)
    return lasttime

BF(pandas.read_csv('2008-08-04_d.csv', index_col=0), 10, 100)